Cuidado com o gerenciador de contexto do PyODBC: ele não fecha conexões.

[Link](https://github.com/mkleehammer/pyodbc/issues/43#issuecomment-1046678648)

In [1]:
import warnings
import pandas as pd
from db_manager import DBManager

DB_SQLITE = "data/chinook.db"

db = DBManager(driver="SQLite3", database=DB_SQLITE)

query_all_tables = "SELECT name FROM sqlite_schema WHERE type='table'"
query_first_10_albums = "SELECT * FROM albums LIMIT 10"

with db:
    tables = [item[0] for item in db.query(query_all_tables)]
    first_10_albums = db.query(query_first_10_albums)

    # beware of the warning supression!! Pandas was only tested with SQLAlchemy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        all_tables_as_dataframes = {
            table: pd.read_sql_query(f"SELECT * from {table}", db.connection)
            for table in tables
        }

tables


['albums',
 'sqlite_sequence',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'sqlite_stat1']

In [2]:
first_10_albums

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8)]

In [3]:
all_tables_as_dataframes.keys()

dict_keys(['albums', 'sqlite_sequence', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks', 'sqlite_stat1'])

In [4]:
for dataframe in all_tables_as_dataframes:
    print(dataframe, all_tables_as_dataframes[dataframe].columns)

albums Index(['AlbumId', 'Title', 'ArtistId'], dtype='object')
sqlite_sequence Index(['name', 'seq'], dtype='object')
artists Index(['ArtistId', 'Name'], dtype='object')
customers Index(['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City',
       'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email',
       'SupportRepId'],
      dtype='object')
employees Index(['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo',
       'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country',
       'PostalCode', 'Phone', 'Fax', 'Email'],
      dtype='object')
genres Index(['GenreId', 'Name'], dtype='object')
invoices Index(['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress',
       'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode',
       'Total'],
      dtype='object')
invoice_items Index(['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity'], dtype='object')
media_types Index(['MediaTypeId', 'Name'], dtype='object')
playlists

In [5]:
all_tables_as_dataframes["customers"].head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisgoncalves@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [6]:
all_tables_as_dataframes["albums"].head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [7]:
db.database

'data/chinook.db'

In [8]:
from pathlib import Path
from sqlalchemy import create_engine, text

engine = create_engine(f"sqlite:///{db.database}")

with engine.connect() as conn:
    all_tables_as_dataframes_sqlachemy = {
        table: pd.read_sql(text(f"SELECT * from {table}"), con=conn)
        for table in tables
            }


In [9]:
all_tables_as_dataframes_sqlachemy.keys()

dict_keys(['albums', 'sqlite_sequence', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks', 'sqlite_stat1'])

In [10]:
all_tables_as_dataframes_sqlachemy["artists"].head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
